In [16]:
import random
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm
import os

PALLETE = {(0, 0, 142) : 0,
 (45, 60, 150) : 1,
 (70, 70, 70) : 2,
 (70, 130, 180) : 3,
 (81, 0, 81) : 4,
 (100, 40, 40) : 5,
 (102, 102, 156) : 6,
 (107, 142, 35) : 7,
 (110, 190, 160) : 8,
 (128, 64, 128) : 9,
 (145, 170, 100) : 10,
 (150, 100, 100) : 11,
 (153, 153, 153) : 12,
 (157, 234, 50) : 13,
 (170, 120, 50) : 14,
 (220, 220, 0) : 15,
 (230, 150, 140) : 16,
 (244, 35, 232) : 17,
 (250, 170, 30) : 18}

def iSAID_convert_from_color(arr_3d, palette=PALLETE):
    """RGB-color encoding to grayscale labels."""
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)
    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i
    return arr_2d

PATH = f'../data/town/'

new_dataset_path = os.path.join('../data', 'formatted_dataset')
os.mkdir(new_dataset_path)

img_dir = os.path.join(new_dataset_path, 'img_dir')
ann_dir = os.path.join(new_dataset_path, 'ann_dir')

os.mkdir(img_dir)
os.mkdir(ann_dir)

new_dataset_train_path_img = os.path.join(img_dir, 'train')
new_dataset_test_path_img = os.path.join(img_dir, 'test')
new_dataset_val_path_img = os.path.join(img_dir, 'val')
os.mkdir(new_dataset_train_path_img)
os.mkdir(new_dataset_test_path_img)
os.mkdir(new_dataset_val_path_img)


new_dataset_train_path_ann = os.path.join(ann_dir, 'train')
new_dataset_test_path_ann = os.path.join(ann_dir, 'test')
new_dataset_val_path_ann = os.path.join(ann_dir, 'val')
os.mkdir(new_dataset_train_path_ann)
os.mkdir(new_dataset_test_path_ann)
os.mkdir(new_dataset_val_path_ann)

towns_path = os.listdir(PATH)

for town_name in towns_path:
    town_path = os.path.join(PATH, town_name)
    waypoints = os.listdir(town_path)
    with tqdm(total=len(waypoints)) as t:
        for waypoint in waypoints:
            waypoint_path = os.path.join(town_path, waypoint)
            images_path = os.path.join(waypoint_path, 'rgb')
            masks_path = os.path.join(waypoint_path, 'seg')
            image_list = os.listdir(images_path)
            mask_list = os.listdir(masks_path)
            for image in image_list:
                train_test_val = random.randint(0, 10)
                new_image_name = image.replace('_rgb', '')
                new_image_name = new_image_name.replace('.', '_img.')

                if train_test_val <= 7:
                    copy_path_img = new_dataset_train_path_img
                    copy_path_ann = new_dataset_train_path_ann
                elif train_test_val <= 9:
                    copy_path_img = new_dataset_test_path_img
                    copy_path_ann = new_dataset_test_path_ann
                else:
                    copy_path_img = new_dataset_val_path_img
                    copy_path_ann = new_dataset_val_path_ann

                shutil.copy2(os.path.join(images_path, image), os.path.join(copy_path_img, new_image_name))
                seg_file = image.replace('rgb', 'seg')
                seg_file = seg_file.replace('.', '_color.')
                new_seg_name = new_image_name.replace('_img', '_mask')
                mask = Image.open(os.path.join(masks_path, seg_file))
                mask = np.array(mask)[:, :, :3]
                mask = iSAID_convert_from_color(mask)
                Image.fromarray(mask).save(os.path.join(copy_path_ann, new_seg_name))

            t.set_description_str(f'Town: {town_name} | Waypoint: {waypoint}')
            t.update(1)


Town: Town03 | Waypoint: waypoint_99: 100%|██████████| 169/169 [29:17<00:00, 10.40s/it]
Town: Town05 | Waypoint: waypoint_110:  10%|▉         | 15/156 [02:44<25:44, 10.95s/it]


KeyboardInterrupt: 

In [17]:
from mmcv import Config
cfg = Config.fromfile('config.py')

In [18]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
import mmcv
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'flow.ipynb'
# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True,
                meta=dict())

2023-02-06 11:36:13,941 - mmseg - INFO - Loaded 1579 images
C:\MySpace\Programs\anaconda3\envs\cvenv\lib\site-packages\mmseg\models\backbones\resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
C:\MySpace\Programs\anaconda3\envs\cvenv\lib\site-packages\mmseg\models\losses\cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
2023-02-06 11:36:14,356 - mmseg - INFO - Loaded 446 images
2023-02-06 11:36:14,357 - mmseg - INFO - Start running, host: Nitcu@Inchore, work_dir: C:\MySpace\Projects\Roads\mmdetection_flow
2023-02-06 11:36:14,358 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 8.00 GiB total capacity; 7.21 GiB already allocated; 0 bytes free; 7.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
from PIL import Image
import numpy as np
from tqdm import tqdm

PATH = f'../data/town/'
towns_path = os.listdir(PATH)

labels = set()
for town_name in towns_path:
    town_path = os.path.join(PATH, town_name)
    waypoints = os.listdir(town_path)
    with tqdm(total=len(waypoints)) as t:
        for waypoint in waypoints:
            waypoint_path = os.path.join(town_path, waypoint)
            masks_path = os.path.join(waypoint_path, 'seg')
            mask_list = os.listdir(masks_path)
            for mask in mask_list:
                if 'color' not in mask:
                    continue

                image = Image.open(os.path.join(masks_path, mask))
                image = np.array(image)
                for i in range(image.shape[0]):
                    for j in range(image.shape[1]):
                        cs = image[i, j]
                        labels.add(tuple(cs)[:3])

            t.set_description_str(f'Town: {town_name} | Waypoint: {waypoint}')
            t.set_postfix_str(f'Count: {len(labels)}')
            t.update(1)

Town: Town03 | Waypoint: waypoint_167:  46%|████▌     | 77/169 [34:44<41:30, 27.07s/it, Count: 19]  


KeyboardInterrupt: 

In [11]:
labels

{(0, 0, 142),
 (45, 60, 150),
 (70, 70, 70),
 (70, 130, 180),
 (81, 0, 81),
 (100, 40, 40),
 (102, 102, 156),
 (107, 142, 35),
 (110, 190, 160),
 (128, 64, 128),
 (145, 170, 100),
 (150, 100, 100),
 (153, 153, 153),
 (157, 234, 50),
 (170, 120, 50),
 (220, 220, 0),
 (230, 150, 140),
 (244, 35, 232),
 (250, 170, 30)}